<a href="https://colab.research.google.com/github/lbk209/topic_modeling/blob/main/llama2_example_kor_nli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QLoRA를 활용한 한국어 LLM Fine-Tuning: Train LLaMA2 7B as a Text Classifier

## 0. 라이브러리 설치

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl

## 1. 데이터셋 준비

- KLUE의 NLI 데이터셋을 사용합니다.

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from dataclasses import dataclass, field
from typing import Optional
import torch
from peft import LoraConfig
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer, pipeline
from trl import SFTTrainer

tqdm.pandas()

nli_data = load_dataset('klue', 'nli')

train_data = nli_data['train']
validation_data = nli_data['validation']

- 데이터셋을 생성모델의 학습 형식에 맞게 변형합니다.

In [ ]:
id_to_label = {0:'entailment', 1:'neutral', 2:'contradiction'}

question_template = "### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. "

train_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(train_data['premise'],train_data['hypothesis'],train_data['label'])]
validation_instructions = [f'{question_template}\npremise: {x}\nhypothesis: {y}\n\n### Assistant: {id_to_label[z]}' for x,y,z in zip(validation_data['premise'],validation_data['hypothesis'],validation_data['label'])]

ds_train = Dataset.from_dict({"text": train_instructions})
ds_validation = Dataset.from_dict({"text": validation_instructions})
instructions_ds_dict = DatasetDict({"train": ds_train, "eval": ds_validation})

In [ ]:
instructions_ds_dict['train']['text'][0]

'### Human: 다음 두 문장의 관계를 entailment, neutral, contradiction 중 하나로 분류해줘. \npremise: 힛걸 진심 최고다 그 어떤 히어로보다 멋지다\nhypothesis: 힛걸 진심 최고로 멋지다.\n\n### Assistant: entailment'

## 2. 모델 학습

- Beomi님께서 한국어 데이터셋으로 추가 훈련 하신 LLaMA2 7B 모델을 Fine-tuning 하겠습니다.
- 우선, 학습에 필요한 Arguments를 준비합니다.

In [ ]:
model_name = "beomi/llama-2-ko-7b"


@dataclass
class ScriptArguments:
    model_name: Optional[str] = field(default=model_name, metadata={"help": "the model name"})
    dataset_text_field: Optional[str] = field(default="text", metadata={"help": "the text field of the dataset"})
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    batch_size: Optional[int] = field(default=4, metadata={"help": "the batch size"})
    seq_length: Optional[int] = field(default=512, metadata={"help": "Input sequence length"})
    gradient_accumulation_steps: Optional[int] = field(
        default=2, metadata={"help": "the number of gradient accumulation steps"}
    )
    load_in_8bit: Optional[bool] = field(default=False, metadata={"help": "load the model in 8 bits precision"})
    load_in_4bit: Optional[bool] = field(default=True, metadata={"help": "load the model in 4 bits precision"})
    use_peft: Optional[bool] = field(default=True, metadata={"help": "Wether to use PEFT or not to train adapters"})
    trust_remote_code: Optional[bool] = field(default=True, metadata={"help": "Enable `trust_remote_code`"})
    output_dir: Optional[str] = field(default="output", metadata={"help": "the output directory"})
    peft_lora_r: Optional[int] = field(default=64, metadata={"help": "the r parameter of the LoRA adapters"})
    peft_lora_alpha: Optional[int] = field(default=16, metadata={"help": "the alpha parameter of the LoRA adapters"})
    logging_steps: Optional[int] = field(default=1, metadata={"help": "the number of logging steps"})
    use_auth_token: Optional[bool] = field(default=False, metadata={"help": "Use HF auth token to access the model"})
    num_train_epochs: Optional[int] = field(default=3, metadata={"help": "the number of training epochs"})
    max_steps: Optional[int] = field(default=-1, metadata={"help": "the number of training steps"})
    save_steps: Optional[int] = field(
        default=100, metadata={"help": "Number of updates steps before two checkpoint saves"}
    )
    save_total_limit: Optional[int] = field(default=10, metadata={"help": "Limits total number of checkpoints."})
    push_to_hub: Optional[bool] = field(default=False, metadata={"help": "Push the model to HF Hub"})
    hub_model_id: Optional[str] = field(default=None, metadata={"help": "The name of the model on HF Hub"})


script_args = ScriptArguments()

- quantization을 적용하여 모델을 불러옵니다.

In [ ]:
if script_args.load_in_8bit and script_args.load_in_4bit:
    raise ValueError("You can't load the model in 8 bits and 4 bits at the same time")
elif script_args.load_in_8bit or script_args.load_in_4bit:
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=script_args.load_in_8bit, load_in_4bit=script_args.load_in_4bit
    )
    device_map = {"": 0}
    torch_dtype = torch.bfloat16
else:
    device_map = None
    quantization_config = None
    torch_dtype = None

model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=quantization_config,
    device_map=device_map,
    trust_remote_code=script_args.trust_remote_code,
    torch_dtype=torch_dtype,
    use_auth_token=script_args.use_auth_token,
)

- LoRA를 적용하여 Trainer를 구성합니다.

In [ ]:
dataset = instructions_ds_dict

training_args = TrainingArguments(
    output_dir=script_args.output_dir,
    per_device_train_batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
    learning_rate=script_args.learning_rate,
    logging_steps=script_args.logging_steps,
    num_train_epochs=script_args.num_train_epochs,
    max_steps=script_args.max_steps,
    report_to=script_args.log_with,
    save_steps=script_args.save_steps,
    save_total_limit=script_args.save_total_limit,
    push_to_hub=script_args.push_to_hub,
    hub_model_id=script_args.hub_model_id,
)

if script_args.use_peft:
    peft_config = LoraConfig(
        r=script_args.peft_lora_r,
        lora_alpha=script_args.peft_lora_alpha,
        bias="none",
        task_type="CAUSAL_LM",
    )
else:
    peft_config = None

trainer = SFTTrainer(
    model=model,
    args=training_args,
    max_seq_length=script_args.seq_length,
    train_dataset=dataset['train'],
    eval_dataset=dataset['eval'],
    dataset_text_field=script_args.dataset_text_field,
    peft_config=peft_config,
)

- 학습을 진행합니다.
- Colab의 V100 Gpu 기준 약 8시간이 소요됩니다.

In [ ]:
trainer.train()

trainer.save_model(training_args.output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.


Map:   0%|          | 0/24998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,4.810600
2,4.898000
3,4.668000
4,4.494500
5,4.724700
6,4.512600
7,4.511700
8,4.924900
9,4.377400
10,4.655300


- 학습한 모델의 성능을 테스트 해 봅니다.

In [ ]:
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "text-generation",
    model=model,|
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map={'':0},
)

In [ ]:
query = instructions_ds_dict['eval']['text'][1].split('### Assistant: ')[0] + '### Assistant:'
queries = [instructions_ds_dict['eval']['text'][i].split('### Assistant: ')[0] + '### Assistant:' for i in range(len(instructions_ds_dict['eval']))]
sequences = pipeline(
    queries,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=3,
    early_stopping=True,
    # do_sample=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
results = []

for seq in sequences:
  result = seq[0]['generated_text'].split('### Assistant:')[1]
  results.append(result)

labels = []

for label in instructions_ds_dict['eval']['text']:
  result = label.split('### Assistant:')[1]
  labels.append(result)

print("Accuracy: ", (len([1 for x, y in zip(results, labels) if y in x]) / len(labels)))

Accuracy:  0.5973333333333334
